In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../data/data.csv")

In [ ]:
df

In [ ]:
import numpy as np
import base64
import matplotlib.pyplot as plt


In [ ]:
def decode_frame_response(response: dict) -> np.ndarray:
    """Decode frame response with shape info."""
    
    # Extract metadata
    width = 150     # 150
    height = 1    # 1
    channels = 3 # 3
    
    # Decode base64
    decoded_bytes = base64.b64decode(response)
    
    # Reshape correctly
    arr = np.frombuffer(decoded_bytes, dtype=np.uint8)
    
    if channels == 3:
        # RGB image
        img = arr.reshape((height, width, channels))
        # img.shape = (1, 150, 3)
    else:
        # Grayscale
        img = arr.reshape((height, width))
        # img.shape = (1, 150)
    
    return img


In [ ]:
bs_image = "WcdkObl2JKqCRb9vYspfS8JsW8hid9BSd9BSdNBUfNJPhtRJftJOV8ZlXslhftJOfNJPZ8xcT8NpO7p1MrV6MLR6UcRol9g+pdo1mtg8l9g+g9NLZ8xcYMlgYspfacxbedFRiNVHiNVHdNBUV8ZlUcRoU8VnUcRoWcdkacxbcM5Wbc5YYspfMLR6H5KMJ3yOKnWOJ32OHpqJNbd4QL1yKa9/H5WLIJCMHpqJJ62AQr5xVcZmMrV6IaaFIaeENbd4U8VnRL5wR8Bua81ZgdNMiNVHfNJPYspfScFtTcJrYspfi9VGi9VGcM5WWcdkScFtRL5wZ8xcldc/ndk6i9VGcM5WfNJPd9BSRb9vNbd4Obl2W8hifNJPg9NLhtRJedFRWcdkMrV6H6OGMLR6YMlgcs9VfNJPhtRJiNVHgdNMacxba81Zcs9VXslhRL5wMrV6MrV6O7p1U8VncM5Wg9NLdNBUTcJrJqyBH5SLJIaNIYyNHpmKIKSFK7F9OLl2OLl2PrxzT8NpQL1yPrxzZ8xcgdNMftJOYspfS8JsRL5wU8VnW8hiNbd4I6iDIqeEM7Z5ScFtObl2QL1yU8VnO7p1K7F9"

In [ ]:
a = decode_frame_response(bs_image)

In [ ]:
a.shape

In [ ]:
plt.figure(figsize=(15, 2))
plt.imshow(a.reshape(1, 150, 3))  # Reshape for visualization
plt.title(f"Depth:, Colormap: viridis")
plt.axis('off')
plt.show()



In [ ]:
import matplotlib.cm as cm


In [ ]:
def apply_colormap(
    grayscale_array: "NDArray[np.uint8]", colormap_name: "viridis"
) -> "NDArray[np.uint8]":
    """Apply colormap to grayscale array, converting to RGB.

    Args:
        grayscale_array: 1D array of grayscale pixel values (0-255)
        colormap_name: Name of colormap to apply

    Returns:
        NDArray[np.uint8]: 3D array of RGB values (height, width, 3)

    Example:
        >>> arr = np.array([0, 128, 255], dtype=np.uint8)
        >>> rgb = apply_colormap(arr, "viridis")
        >>> rgb.shape
        (1, 3, 3)
    """
    # Normalize to [0, 1] range
    normalized = grayscale_array.astype(np.float32) / 255.0

    # Get colormap
    colormap = cm.get_cmap(colormap_name)

    # Apply colormap - colormap expects scalar or array, returns (N, 4) for array
    rgba = colormap(normalized)

    # Handle both scalar and array cases
    if rgba.ndim == 1:
        rgba = rgba.reshape(1, -1)

    # Convert to uint8 and extract RGB (drop alpha)
    rgb = (rgba[:, :3] * 255).astype(np.uint8)

    # Reshape to (height, width, 3) for single row
    height = 1
    width = len(grayscale_array)
    rgb_reshaped = rgb.reshape((height, width, 3))

    return rgb_reshaped



In [ ]:
import matplotlib.pyplot as plt
decoded = decode_bs_image(bs_image)  # Önceki fonksiyonla çözülmüş verin

colored = apply_colormap(decoded.flatten(), "viridis")
colored = colored.reshape(decoded.shape[0], decoded.shape[1], 3)
plt.imshow(colored)
plt.title("Colormapped Depth Image")
plt.axis('off')
plt.show()


In [ ]:
decoded.shape

In [ ]:
import sys
sys.path.append("/home/yigit/Desktop/case_study/AIQ/image-api/src")






In [ ]:
from image_api.clients.database import db_client
from image_api.config.logging_config import get_logger, log_with_extra
from image_api.models.schemas import FrameQueryParams, FrameResponse
from image_api.utilities.compression import decompress_array
from image_api.utilities.database_ops import get_frames_buffered
from image_api.utilities.image_processing import (
    apply_colormap,
    encode_rgb_to_base64,
)
